# Project 2 : two dimentional Ising model

In [ ]:
import pickle
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn.model_selection as skms
import sklearn.linear_model as skl
import sklearn.metrics as skm
import tqdm
import copy
import time
from IPython.display import display

%matplotlib inline

sns.set(color_codes=True)

In [ ]:
file_dir = os.path.join(os.getcwd(), "files")
filenames = glob.glob(os.path.join(file_dir, 'Ising2D*'))
label_filename = list(filter(lambda x: "label" in x, filenames))[0]
dat_filename = list(filter(lambda x: "label" not in x, filenames))[0]

# Read in the labels
with open(label_filename, "rb") as f:
    labels = pickle.load(f)

# Read in the corresponding configurations
with open(dat_filename, "rb") as f:
    data = np.unpackbits(pickle.load(f)).reshape(-1, 1600).astype("int")

# Set spin-down to -1
data[data == 0] = -1

In [ ]:
import pickle
import numpy as np
def read_t(t,root="./files/"):
    data = pickle.load(open(root+'Ising2DFM_reSample_L40_T=%.2f.pkl'%t,'rb'))
    return np.unpackbits(data).astype(int).reshape(-1,1600)

### Excluding T = 2.0, 2.25, 2.5 because they are the critical phase and we have a binary model.

In [ ]:
t = [0.25, 0.5, 0.75, 1.0, 1.25, 1.5, 1.75, 2.75, 3.0, 3.25, 3.5, 3.75, 4.0]

ordered = []
disordered = []

for t in t:
    if (t<2.0):
        ordered.append(read_t(t))
    else:
        disordered.append(read_t(t))

In [ ]:
# set spin-down to minus one. 
ordered[ordered == 0] = -1
disordered[disordered == 0] = -1

# now the spin has values either plus or minus one.

In [ ]:
# Labels for data

In [ ]:
cwd = os.getcwd()
filenames = glob.glob(os.path.join(cwd, 'files*'))

In [ ]:
# label_filename = list(filter(lambda x: "label" in x, filenames))[0]
# dat_filename = list(filter(lambda x: "label" not in x, filenames))[0]

# Read in the labels
with open(label_filename, "rb") as f:
    labels = pickle.load(f)

# Read in the corresponding configurations
with open(dat_filename, "rb") as f:
    data = np.unpackbits(pickle.load(f)).reshape(-1, 1600).astype("int")

# Set spin-down to -1
data[data == 0] = -1